# Quick Start
___

这是一个简单示例，以最小的操作步骤，来体验框架的基础功能。

## 安装框架
___

AutoGan requires Python version >= 3.8. It can be installed from pip:

In [ ]:
%%capture --no-stderr
%pip install pyautogan

In [ ]:
import autogan

## LLM 配置
___

In [ ]:
# 通过 dict_from_json 函数读取 LLM 配置
llm_config_dict = autogan.dict_from_json("LLM_CONFIG")

dict_from_json 该函数会优先从环境变量 "LLM_CONFIG" 中获取配置，如环境变量中没有找到 "LLM_CONFIG"，则会从当前目录中名为 "LLM_CONFIG" 的文件内读取配置。

对于配置内容，已在当前目录下已放置了 LLM_CONFIG 文件示例，其中 main_model 为 agent 智能体的 LLM 配置，而 summary_model 为用于压缩上下文或其他文本内容的 LLM 配置。
summary_model 配置为可选项，当缺失时框架会使用 main_model 的配置进行上下文或文本压缩。

LLM 配置中的 api_key_list 是一个配置列表。在请求时，系统将按照列表顺序进行尝试，当前请求失败，系统将自动切换至下一个配置进行尝试。
当列表中的第一个配置请求失败后，会使用下一个配置继续尝试。

**注意：**  框架目前仅适配了 openai、azure、自定义类 openai 接口的 chat 模型。

## 构建 Agents
___

In [ ]:
customer = autogan.HumanAgent("客户")
customer_manager = autogan.UniversalAgent("客户经理", duty="Responsible for receiving customers", work_flow="我希望你在群聊中成为高端服务公司的金牌客户经理, 尽一切可能满足客户的合法需求, 在满足客户的要求之前不放弃.注意:如果客户提供文档，优先考虑从文档中寻找答案。再次强调，完成客户的任务是非常重要的，尽一切可能去完成它，发挥你的想象力，不要轻易放弃。如果没有正确答案，就不要给出错误的答案。还有不用解释自己的计划。")
coder = autogan.UniversalAgent("程序员", duty="I can accomplish tasks previously impossible for you by writing programs.", work_flow="我希望你是一个有经验的Python程序员。在编写代码之前，请做好以下准备工作:如果生成了代码，您必须在回复内容的开头@代码测试专员。这很重要。输出代码必须用' ' '包装。当程序运行缺少依赖时，你可以编写安装依赖的语句后 @代码测试专员 来安装相关依赖")
code_execution_agent = autogan.ToolAgentCodeExecution("代码测试专员")

## 构建组织架构
___

每个 list 相当于一个部门，各部门的第一个 agent 为部门的 leader，负责与上级部门的 agent 进行沟通。
因此 leader 的 duty 属性应当概述整个部门的职测与能力。

In [ ]:
org_structure = [customer, customer_manager, coder, code_execution_agent]

## 开始会话
___

与 agent 的初次对话，需要在发送的消息中添加 task_tag 符号，用于创建任务，task_tag 的默认值为 "/task"。

后续对话中如果使用 task_tag 符号，则是对当前任务发表意见或看法。

In [ ]:
# 参数：
# 1.组织架构
# 2.邀请第一个发言的 agent 名称
# 3.更改 task_tag 的值，默认为 "/task"
autogan.AgentSwitch(org_structure, "/任务", customer, llm_config_dict);